In [1]:
import numpy as np 
from matplotlib import pyplot as plt 
import os
from tqdm import tqdm

from read_binary import read_binary    

In [2]:
files = sorted(os.listdir("simulation"), key=lambda x:float(x.split()[0]))

In [3]:
paths = [os.path.join("simulation", x) for x in files]

radius = 0.2
ts, rss, vss = [], [], []
for p in tqdm(paths):
    _,t,_,rs, vs = read_binary(p)
    ts.append(t)
    rss.append(rs)
    vss.append(vs)
ts = np.array(ts)
rss = np.array(rss)
vss = np.array(vss)

100%|██████████| 1172/1172 [00:01<00:00, 1079.85it/s]


In [4]:
from mpl_toolkits.mplot3d import Axes3D

def plot_shere(ax, x, y, z, r, resolution=100, **kwargs):
    """ simple function to plot a sphere to a 3d axes """

    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)

    xx = r * np.outer(np.cos(u), np.sin(v)) + x
    yy = r * np.outer(np.sin(u), np.sin(v)) + y
    zz = r * np.outer(np.ones(np.size(u)), np.cos(v)) + z

    ax.plot_surface(xx, yy, zz,  rstride=4, cstride=4, **kwargs)


In [8]:
for i in tqdm(range(len(paths))):
    rs = rss[i]
    colors = plt.rcParams["axes.prop_cycle"][:len(rs)]

    # set up the figure
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # loop through the data and plot the spheres
    for pos, c in zip(rs, colors):
        plot_shere(ax, *pos, radius, edgecolor='none', color=c["color"])

    # set the axes limits and show the plot
    ax.set_ylim([0,1])
    ax.set_xlim([0,1])
    ax.set_zlim([0,1])
    plt.savefig(os.path.join("imgs", f"{i:04d}.png"))

    # plt.show()


  8%|▊         | 94/1112 [00:56<12:35,  1.35it/s]

KeyboardInterrupt: 